# Neural Net From Scratch

### Imports

In [8]:
import numpy as np

np.random.seed(1)

### Activation functions

In [9]:
class Linear():
    @staticmethod
    def f(x):
        return x
    @staticmethod
    def df(x):
        return x

class Relu():
    @staticmethod
    def f(x):
        return x * (x > 0)
    @staticmethod
    def df(x):
        return 1 * (x > 0)

class Sigmoid():
    @staticmethod
    def f(x):
        return 1.0 / (1.0 + np.exp(-x))
    @staticmethod
    def df(x):
        return Sigmoid.f(x) * (1 - Sigmoid.f(x))

### Loss functions

In [32]:
def MSE(target, output):
    return np.mean((target - output) **2)

### Dense layer

In [11]:
class Layer:
    def __init__(self, neurons, activation=Linear):
        self.neurons = neurons
        self.activation = activation
        self.weights = None
        self.biases = None
        self.lr = 0.02 # Snelheid waarmee de model traint -> staat nu hardcoded

    def initialize(self, prev_neurons):
        self.weights = np.random.normal(0.0, 1.0, (self.neurons, prev_neurons))
        self.biases = np.random.normal(size=(self.neurons, 1))

    def feed_forward(self, input_array):
        # Input en output opslaan -> zijn nodig voor gradient descent
        self.inputs = input_array 
        
        # Y = W * I + B (matrix * vector)
        self.output = self.activation.f(np.dot(self.weights, self.inputs) + self.biases) 
        return self.output

    def gradient_descent(self, errors):
        delta_weights = self.lr * np.dot((errors * self.activation.df(self.output)), self.inputs.T)
        delta_biases = self.lr * errors * self.activation.df(self.output)

        self.weights = np.add(self.weights, delta_weights)
        self.biases = np.add(self.biases, delta_biases)

### Neurale netwerk

In [33]:
class NeuralNetwork():
    def __init__(self):
        self.layers = []

    def add(self, layer):
        # Initialiseer de laag met weights en biases alleen als het niet de input laag is.
        if len(self.layers) > 0:
            prev_neurons = self.layers[-1].neurons
            layer.initialize(prev_neurons)
        
        self.layers.append(layer)
        

    def predict(self, input_array, transposed = False):
        output = input_array
        if not transposed:
            output = output.T

        for lay in self.layers[1:]:
            output = lay.feed_forward(output)
        return output

    def fit(self, X_values ,y_true, epochs=10):          
        for epoch in range(epochs):
            sum_errors = 0
            for x, y in zip(X_values, y_true):
                # feedforward
                prediction = self.predict(x.reshape(-1,1), transposed = True) # Reshape maatkt ndmin 2 en transposed het al
                
                # bereken error
                error = (y - prediction)
                sum_errors += MSE(y, prediction) # houdt mse bij om het te visualiseren
                
                # backward propagation
                # error = mse_derivative(y, prediction)
                for lay in reversed(self.layers[1:]):
                    lay.gradient_descent(error)
                    error = np.dot(lay.weights.T, error)
            sum_errors /= len(X_values)
            print(f"Epoch {epoch+1}/{epochs}:")
            print(f"Error: {sum_errors}\n")
        print(f"\nFinished Training\n{'=' * 50}")



### Train

Getallen optellen

e.g. array van [0.23, 0.56] = 0.79

In [36]:
from random import random

nn = NeuralNetwork()

nn.add(Layer(2, activation=Relu))
nn.add(Layer(3, activation=Relu))
nn.add(Layer(1, activation=Relu))

X = np.array([[random() for _ in range(2)] for _ in range(1000)])
y = np.array([[i[0] + i[1]] for i in X])

nn.fit(X, y, epochs=5)

Epoch 1/5:
Error: 0.11088301575230647

Epoch 2/5:
Error: 0.0070471463506549484

Epoch 3/5:
Error: 0.00017965201895630005

Epoch 4/5:
Error: 4.8942662358929857e-05

Epoch 5/5:
Error: 2.4816648280527504e-05


Finished Training


In [37]:
test_set = np.array([[0.4, 0.4], [0.27, 0.32], [0.1, 0.13], [0.76, 0.22]]) # 0.8, 0.59, 0.23,  0.98
nn.predict(test_set)

array([[0.7996045 , 0.58942643, 0.22923837, 0.98030834]])